In [28]:
import httpx
from tqdm import tqdm
import os
from dotenv import load_dotenv
import json
from pprint import pprint

In [18]:
"""Use this to check the data that is being extracted from the discourse"""

with open("./content/content_0.json", "r") as f:
    data = json.load(f)


In [19]:
print(data.keys())
print(data['topic_list'].keys())

dict_keys(['users', 'primary_groups', 'flair_groups', 'topic_list'])
dict_keys(['can_create_topic', 'more_topics_url', 'per_page', 'top_tags', 'topics'])


In [20]:
for key in data['topic_list'].keys():
    print(type(data['topic_list'][key]), key)

<class 'bool'> can_create_topic
<class 'str'> more_topics_url
<class 'int'> per_page
<class 'list'> top_tags
<class 'list'> topics


In [21]:
print(data['topic_list']['more_topics_url']) # This is indicating the next URL to go to. whenever is required

/c/courses/tds-kb/34?page=1


In [30]:
# upto this point we have gotten where we can find out the topics
for x in data['topic_list']['topics']:
    print(x)
    break

{'id': 176077, 'title': 'Project1 - Virtual TA - Discussion Thread [TDS May 2025]', 'fancy_title': 'Project1 - Virtual TA - Discussion Thread [TDS May 2025]', 'slug': 'project1-virtual-ta-discussion-thread-tds-may-2025', 'posts_count': 236, 'reply_count': 151, 'highest_post_number': 242, 'image_url': None, 'created_at': '2025-05-22T21:36:32.840Z', 'last_posted_at': '2025-06-11T19:43:11.008Z', 'bumped': True, 'bumped_at': '2025-06-11T19:43:11.008Z', 'archetype': 'regular', 'unseen': False, 'last_read_post_number': 242, 'unread': 0, 'new_posts': 0, 'unread_posts': 0, 'pinned': True, 'unpinned': None, 'excerpt': 'Please post any questions related to Project 1 \nPlease use markdown code formatting (fenced code blocks beginning with ```) when sharing code (rather than screenshots). It’s easier for us to copy-paste and test. \nDeadlin&hellip;', 'visible': True, 'closed': False, 'archived': False, 'notification_level': 2, 'bookmarked': False, 'liked': False, 'tags': ['term2-2025', 'tds-projec

What is data:
> data => what is obtained through `https://discourse.onlinedegree.iitm.ac.in/c/courses/tds-kb/34.json?page=n` for each page separate value it takes
**Important findins from the `data`**
- `topic_list` is the only useful key --> contains the `topics` and the `more_topics_url` (pointing to the next page to find topics)

**Important findings from the `data['topic_list']['topics']`**
- slug --> indicates the url directions as `/t/{slug}`
- id --> append the id `/t/{slug}/{id}`
- posts_count --> loop through the `/t/{slug}/{id}/{1..posts_count}`
- created at --> initial date of creation for the topic (between range)
- last_posted_at --> final post date in the topic (between range)

This should be good enough for us to scrape the posts content
---
 - convert the json to markdown based on content from each post (Images can be tricky)
 - orchestrate the code across range of dates
---
Finally if everything got right till now. Then we can safely embed and build the RAG

In [33]:
with open('./content/post.json', 'r') as f:
    posts = json.load(f)

In [34]:
posts

[{'id': 629634,
  'name': 'Jivraj Singh Shekhawat',
  'username': 'Jivraj',
  'avatar_template': 'https://avatars.discourse-cdn.com/v4/letter/j/b9bd4f/{size}.png',
  'created_at': '2025-05-22T21:36:33.118Z',
  'cooked': '<p>Please post any questions related to <a href="https://tds.s-anand.net/#/project-tds-virtual-ta" rel="noopener nofollow ugc">Project 1</a></p>\n<p>Please use markdown code formatting (fenced code blocks beginning with ```) when sharing code (rather than screenshots). It’s easier for us to copy-paste and test.</p>\n<p>Deadline: <span class="discourse-local-date" data-date="2025-06-14" data-email-preview="2025-06-14T18:29:00Z UTC" data-time="23:59:00" data-timezone="Asia/Calcutta">2025-06-14T18:29:00Z</span></p>',
  'post_number': 1,
  'post_type': 1,
  'posts_count': 236,
  'updated_at': '2025-05-22T21:36:33.118Z',
  'reply_count': 0,
  'reply_to_post_number': None,
  'quote_count': 0,
  'incoming_link_count': 121,
  'reads': 470,
  'readers_count': 469,
  'score': 69